# Continuous Control

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
!nvidia-smi

/bin/sh: nvidia-smi: command not found


In [1]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Reacher.app"`
- **Windows** (x86): `"path/to/Reacher_Windows_x86/Reacher.exe"`
- **Windows** (x86_64): `"path/to/Reacher_Windows_x86_64/Reacher.exe"`
- **Linux** (x86): `"path/to/Reacher_Linux/Reacher.x86"`
- **Linux** (x86_64): `"path/to/Reacher_Linux/Reacher.x86_64"`
- **Linux** (x86, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86"`
- **Linux** (x86_64, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86_64"`

For instance, if you are using a Mac, then you downloaded `Reacher.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Reacher.app")
```

In [2]:
env = UnityEnvironment(file_name='Reacher_Linux_NoVis/Reacher.x86_64')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_size -> 5.0
		goal_speed -> 1.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
from unityagents import UnityEnvironment
import numpy as np
env = UnityEnvironment(file_name='Reacher_Linux_NoVis/Reacher.x86_64')
brain_name = env.brain_names[0]
brain = env.brains[brain_name]
env_info = env.reset(train_mode=True)[brain_name

### 2. Examine the State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)
print("episode done ",env_info.local_done)
# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 1
Size of each action: 4
episode done  [False]
There are 1 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726671e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]


In [4]:
import torch
import numpy as np
from collections import deque
from unityagents import UnityEnvironment
from config import Config
from model2 import Actor, Critic
from memory import ReplayBuffer
from noise import OUNoise
from agent1 import DDPGAgent

import matplotlib.pyplot as plt
from unityagents import UnityEnvironment
import numpy as np
env = UnityEnvironment(file_name='Reacher_Linux_NoVis/Reacher.x86_64')
brain_name = env.brain_names[0]
brain = env.brains[brain_name]
env_info = env.reset(train_mode=True)[brain_name]


config = Config()

config.seed = 2
config.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
config.action_size = brain.vector_action_space_size
config.states = env_info.vector_observations
config.state_size = config.states.shape[1]
config.num_agents = len(env_info.agents)

config.actor_hidden_units = (512, 256)
config.actor_learning_rate = 1e-4
config.actor_network_fn = lambda: Actor(config.action_size, config.state_size, config.actor_hidden_units, config.seed).to(config.device)
config.actor_optimizer_fn = lambda params: torch.optim.Adam(params, lr=config.actor_learning_rate)

config.critic_hidden_units = (512, 256)
config.critic_learning_rate = 3e-4
config.weight_decay = 0
config.critic_network_fn = lambda: Critic(config.action_size, config.state_size, config.critic_hidden_units, config.seed).to(config.device)
config.critic_optimizer_fn = lambda params: torch.optim.Adam(params, lr=config.critic_learning_rate)

config.batch_size = 512
config.buffer_size = int(1e6)
config.discount = 0.99
config.update_every = 4
config.memory_fn = lambda: ReplayBuffer(config.action_size, config.buffer_size, config.batch_size, config.seed, config.device)

config.noise_fn = lambda: OUNoise(config.action_size, config.seed)

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_size -> 5.0
		goal_speed -> 1.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


In [5]:
env_info.agents

[922]

In [5]:
agent = DDPGAgent(config)

In [6]:
def ddpg(n_episodes=5000, max_t=2000):
    all_scores = []
    scores_window = deque(maxlen=100)
    
    for i_episode in range(1, n_episodes+1):
        
        agent.reset()
        env_info = env.reset(train_mode=True)[brain_name]
        states = env_info.vector_observations           
        scores = np.zeros(config.num_agents)

        for _ in range(max_t):
            actions = agent.act(states)
            env_info = env.step(actions)[brain_name]
            rewards = env_info.rewards
            next_states = env_info.vector_observations
            dones = env_info.local_done

            agent.step(states, actions, rewards, next_states, dones)

            scores += rewards
            states = next_states
                
        avg_score = np.mean(scores)
        scores_window.append(avg_score)
        all_scores.append(avg_score)
        
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)), end="")
        if i_episode % 100 == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
        if np.mean(scores_window)>=30.0:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_window)))
            torch.save(agent.actor_local.state_dict(), 'checkpoint_actor.pth')
            torch.save(agent.critic_local.state_dict(), 'checkpoint_critic.pth')
            break 
            
    return all_scores

In [7]:
scores = ddpg()

# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

/home_01/f20150198/miniconda2/envs/python3/lib/python3.5/site-packages/torch/nn/functional.py:995: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Episode 3	Average Score: 0.31

KeyboardInterrupt: 

### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [6]:
env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
while True:
    actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    print(rewards)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

[0.0]
[0.0]
[0.0]
[0.0]
[0.0]
[0.0]
[0.0]
[0.0]
[0.0]
[0.0]
[0.0]
[0.0]
[0.0]
[0.0]
[0.0]
[0.0]
[0.0]
[0.0]
[0.0]
[0.0]
[0.0]
[0.0]
[0.0]
[0.0]
[0.0]
[0.0]
[0.0]
[0.0]
[0.0]
[0.0]
[0.0]
[0.0]
[0.0]
[0.0]
[0.0]
[0.0]
[0.0]
[0.0]
[0.0]
[0.0]
[0.0]
[0.0]
[0.0]
[0.0]
[0.0]
[0.0]
[0.0]
[0.0]
[0.0]
[0.0]
[0.0]
[0.0]
[0.0]
[0.0]
[0.0]
[0.0]
[0.0]
[0.0]


KeyboardInterrupt: 

In [1]:
from collections import deque
from ddpg_agent import Agent
from unityagents import UnityEnvironment
import numpy as np
env = UnityEnvironment(file_name='Reacher_Linux_NoVis/Reacher.x86_64')
brain_name = env.brain_names[0]
brain = env.brains[brain_name]
env_info = env.reset(train_mode=True)[brain_name]
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)
print("episode done ",env_info.local_done)
# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
agent = Agent(state_size=state_size, action_size=action_size, random_seed=2)
#scores = np.zeros(1)                          # initialize the score (for each agent)
print("hello")
def ddpg(n_episodes=5000, max_t=2000, print_every=100):
    scores_deque = deque(maxlen=print_every)
    scores = []
    for i_episode in range(1, n_episodes+1):
        #/print("mello")
        env_info = env.reset(train_mode=True)[brain_name]     # reset the environment
        #print("kello")
        state = env_info.vector_observations                 # get the current state (for each agent)
       # print(state.shape,"first")
        score = [0]
       # print("maxt",max_t)
        for t in range(max_t):
            action = agent.act(state)
            print(action)
            env_info = env.step(action)[brain_name]           # send all actions to tne environment
            next_state = env_info.vector_observations         # get next state (for each agent)
#             print(next_state)
            reward = env_info.rewards                        # get reward (for each agent)
#             print(reward)
            done = env_info.local_done                        # see if episode finished
           # print(len(done))
#             print(done)
            agent.step(state, action, reward, next_state, done)
            state = next_state
            print(reward)
            score += reward
           # print(t,"time")
           # print("score",score)
        avg_score = np.mean(score)
        scores_deque.append(avg_score)
        scores.append(avg_score)
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)), end="")
        if np.mean(scores_deque) >= 30.0:
            torch.save(agent.actor_local.state_dict(), 'checkpoint_actor.pth')
            torch.save(agent.critic_local.state_dict(), 'checkpoint_critic.pth')
            break
        if i_episode % print_every == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))
            
    return scores

scores = ddpg()

fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_size -> 5.0
		goal_speed -> 1.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Size of each action: 4
episode done  [False]
hello
[[0.1433571  0.21182206 0.02547583 0.06196284]]
[0.0]
[[0.28165898 0.3304662  0.15777989 0.12106409]]
[0.0]
[[0.35316113 0.40545145 0.25254792 0.14139797]]
[0.0]
[[0.37870696 0.42656898 0.3614611  0.32598722]]
[0.0]
[[0.50384283 0.47476628 0.39850992 0.33766338]]
[0.0]
[[0.4273956  0.41241902 0.4341423  0.35768384]]
[0.0]
[[0.43123418 0.53234977 0.4765858  0.4231666 ]]
[0.0]
[[0.4056642  0.46072778 0.47251487 0.39401457]]
[0.0]
[[0.43882784 0.59475094 0.5389058  0.37823108]]
[0.0]
[[0.54357624 0.6682625  0.60739136 0.509707  ]]
[0.0]
[[0.60645807 0.72936064 0.589432   0.636395  ]]
[0.0]
[[0.69959223 0.65556335 0.65405643 0.6909759 ]]
[0.0]
[[0.6785864  0.6667204  0.65631133 0.77915543]]
[0.0]
[[0.6685098 0.7365291 0.6310544 0.8457528]]
[0.0]
[[0.73984617 0.72178584 0.6523903  0.9099022 ]]
[0.0]
[[0.76533353 0.7144039  0.6015734  0.84541553]]
[0.0]
[[0.782032   0.643972   0.6957501  0.77959126]]
[0.0]
[[0.838928   0.6131705  0.78529125 

/home_01/f20150198/miniconda2/envs/python3/lib/python3.5/site-packages/torch/nn/functional.py:995: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


[0.0]
[[0.7409529  0.5808976  0.7846425  0.88829124]]
[0.0]
[[0.74733645 0.5142567  0.8331797  0.9076777 ]]
[0.0]
[[0.8102697  0.48122698 0.8590978  0.7904566 ]]
[0.0]
[[0.81325203 0.46874255 0.77666295 0.8540531 ]]
[0.0]
[[0.7064181  0.5079763  0.8312063  0.78277177]]
[0.0]
[[0.63755447 0.61189944 0.7915397  0.81645745]]
[0.0]
[[0.5445195  0.5961769  0.70658207 0.83601904]]
[0.0]
[[0.4743866  0.701205   0.6064224  0.86370176]]
[0.0]
[[0.40045312 0.65019304 0.67927015 0.7715731 ]]
[0.0]
[[0.37050876 0.6937445  0.6554372  0.67043674]]
[0.0]
[[0.5078997 0.6232746 0.5644468 0.6444195]]
[0.0]
[[0.54939175 0.68159044 0.5020418  0.6206587 ]]
[0.0]
[[0.4665258 0.6735068 0.5793759 0.6797445]]
[0.0]
[[0.5689952  0.7288696  0.6657643  0.72193027]]
[0.0]
[[0.5704142 0.6680003 0.7004702 0.6803185]]
[0.0]
[[0.49517655 0.65994406 0.7722854  0.60972   ]]
[0.0]
[[0.5277224  0.64232624 0.76135606 0.5493938 ]]
[0.0]
[[0.6291914  0.602329   0.7681615  0.55399907]]
[0.0]
[[0.5303191  0.62656444 0.6802281 

[0.0]
[[0.45370063 0.66363287 0.7343596  0.77720004]]
[0.0]
[[0.4035059 0.7287274 0.6958825 0.8585173]]
[0.0]
[[0.467198   0.7866557  0.76802397 0.8409355 ]]
[0.0]
[[0.58775574 0.6762049  0.71881235 0.8905943 ]]
[0.0]
[[0.49699843 0.70813066 0.8098322  0.90579516]]
[0.0]
[[0.5906999  0.61839104 0.7965996  0.8967821 ]]
[0.0]
[[0.5832513  0.61321384 0.83441496 0.7985285 ]]
[0.0]
[[0.49832568 0.6461415  0.90071684 0.8474945 ]]
[0.0]
[[0.54838914 0.6126234  0.94712454 0.7316787 ]]
[0.0]
[[0.50459325 0.67981744 0.9873097  0.70764846]]
[0.0]
[[0.5977207 0.6020306 0.9613768 0.6198901]]
[0.0]
[[0.5407818  0.5556144  0.82058156 0.6088952 ]]
[0.0]
[[0.5503154  0.5351446  0.82586074 0.53049725]]
[0.0]
[[0.563571   0.56160957 0.78529674 0.64175826]]
[0.0]
[[0.49602282 0.5638357  0.7608103  0.62844443]]
[0.0]
[[0.6091179  0.59588385 0.75440454 0.6299266 ]]
[0.0]
[[0.5953264  0.7002824  0.80698216 0.67399615]]
[0.0]
[[0.5302159  0.71924764 0.7190065  0.6534258 ]]
[0.0]
[[0.44901875 0.7573373  0.8117

KeyboardInterrupt: 

When finished, you can close the environment.

In [ ]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```